# Programming Assignment 3: Information Retrieval (Winter 2026)

In this assignment you will be improving upon a rather poorly-made information retrieval system. You will build an inverted index to quickly retrieve documents that match queries and then make it even better by using term-frequency inverse-document-frequency weighting and cosine similarity to compare queries to your data set. Your IR system will be evaluated for accuracy on the correct documents retrieved for different queries and the correctly computed tf-idf values and cosine similarities.

## Data

You will be using your IR system to find relevant documents among a collection of sixty books sourced from Project Gutenberg. The training data is located in the `data/` directory under the subdirectory `ProjectGutenberg/`. Within this directory you will see yet another directory `raw/`. This contains the raw text files of the sixty books. The `data/` directory also contains the files `dev_queries.txt` and `dev_solutions.txt`. We have provided these to you as a set of development queries and their expected answers to use as you begin implementing your IR system.

## Your Task

Improve upon the given IR system by implementing the following features:

<b>Inverted Index:</b> Implement an inverted index - a mapping from words to the documents in which they occur and the count of how many times each word appears in each document.

<b>TF-IDF:</b> Compute and store the term-frequency inverse-document-frequency value for every word-document co-occurrence.

<b>Cosine Similarity:</b> Implement cosine similarity in order to improve upon the ranked retrieval system, which currently retrieves documents based upon the Jaccard coefficient between the query and each document. Note that when computing $w_{t, q}$ (i.e. the weight for the word 𝑤 in the query) do not include the idf term. That is, $w_{t, q} = 1 + \log_{10} \text{tf}_{t, q}$. Normalize by both the query and document vector lengths.

<b>Dense Retrieval:</b> Use pre-computed BERT embeddings to implement dense retrieval. You will implement cosine similarity for dense vectors and a function to retrieve the top-k most similar documents given a query embedding.

<b>Reflection:</b> Compare TF-IDF and dense retrieval methods, reflecting on their strengths and weaknesses.

## Evaluation

Your IR system will be evaluated on a small development set of queries as well as a held-out set of queries. The development queries are encoded in the file <b>dev_queries.txt</b> and are:

- chest of drawers
- machine learning is cool
- the cold breeze
- pacific coast
- pumpkin pies
- i completed fizzbuzz

We test your system based on five components: the inverted index (term counts), getting postings lists, computing the correct TF-IDF values, implementing cosine similarity using the TF-IDF values, and dense retrieval. The provided development queries contain some common words, some uncommon words, and the occasional non-existent word. Some of the query phrases are found verbatim in the book dataset, and some are not. Your system should be able to correctly handle all of these cases.

### Environment Check

Before we do anything else, let's quickly check that you're running the correct
version of Python and are in the right environment!

In [ ]:
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs124"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 10

If the above cell complains, it means that you're using the wrong environment
or Python version!

If so, please exit this notebook, kill the notebook server with CTRL-C, and
try running

$ `conda activate cs124`

then restarting your notebook server with

$ `jupyter notebook`

If that doesn't work, you should go back and follow the installation
instructions in PA0.

## Getting Started!

We will first start by importing some modules and setting up our IR system class.


In [ ]:
import json
import os
import re
import sys
import math
import numpy as np
from collections import defaultdict
from collections import Counter

from porter_stemmer import PorterStemmer

In [ ]:
class IRSystem:
    def __init__(self):
        # For holding the data - initialized in read_data()
        self.titles = []
        self.docs = []
        self.vocab = []
        # For the text pre-processing.
        self.alphanum = re.compile('[^a-zA-Z0-9]')
        self.p = PorterStemmer()

    def get_uniq_words(self):
        uniq = set()
        for doc in self.docs:
            for word in doc:
                uniq.add(word)
        return uniq

    def __read_raw_data(self, dirname):
        print("Stemming Documents...")

        titles = []
        docs = []
        os.mkdir('%s/stemmed' % dirname)
        title_pattern = re.compile('(.*) \d+\.txt')

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/raw' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        for i, filename in enumerate(filenames):
            title = title_pattern.search(filename).group(1)
            print("    Doc %d of %d: %s" % (i + 1, len(filenames), title))
            titles.append(title)
            contents = []
            f = open('%s/raw/%s' % (dirname, filename), 'r', encoding="utf-8")
            of = open('%s/stemmed/%s.txt' % (dirname, title), 'w',
                      encoding="utf-8")
            for line in f:
                # make sure everything is lower case
                line = line.lower()
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # remove non alphanumeric characters
                line = [self.alphanum.sub('', xx) for xx in line]
                # remove any words that are now empty
                line = [xx for xx in line if xx != '']
                # stem words
                line = [self.p.stem(xx) for xx in line]
                # add to the document's conents
                contents.extend(line)
                if len(line) > 0:
                    of.write(" ".join(line))
                    of.write('\n')
            f.close()
            of.close()
            docs.append(contents)
        return titles, docs

    def __read_stemmed_data(self, dirname):
        print("Already stemmed!")
        titles = []
        docs = []

        # make sure we're only getting the files we actually want
        filenames = []
        for filename in os.listdir('%s/stemmed' % dirname):
            if filename.endswith(".txt") and not filename.startswith("."):
                filenames.append(filename)

        if len(filenames) != 60:
            msg = "There are not 60 documents in ../data/ProjectGutenberg/stemmed/\n"
            msg += "Remove ../data/ProjectGutenberg/stemmed/ directory and re-run."
            raise Exception(msg)

        for i, filename in enumerate(filenames):
            title = filename.split('.')[0]
            titles.append(title)
            contents = []
            f = open('%s/stemmed/%s' % (dirname, filename), 'r',
                     encoding="utf-8")
            for line in f:
                # split on whitespace
                line = [xx.strip() for xx in line.split()]
                # add to the document's conents
                contents.extend(line)
            f.close()
            docs.append(contents)

        return titles, docs

    def read_data(self, dirname):
        """
        Given the location of the 'data' directory, reads in the documents to
        be indexed.
        """
        # NOTE: We cache stemmed documents for speed
        #       (i.e. write to files in new 'stemmed/' dir).

        print("Reading in documents...")
        # dict mapping file names to list of "words" (tokens)
        filenames = os.listdir(dirname)
        subdirs = os.listdir(dirname)
        if 'stemmed' in subdirs:
            titles, docs = self.__read_stemmed_data(dirname)
        else:
            titles, docs = self.__read_raw_data(dirname)

        # Sort document alphabetically by title to ensure we have the proper
        # document indices when referring to them.
        ordering = [idx for idx, title in sorted(enumerate(titles),
                                                 key=lambda xx: xx[1])]

        self.titles = []
        self.docs = []
        numdocs = len(docs)
        for d in range(numdocs):
            self.titles.append(titles[ordering[d]])
            self.docs.append(docs[ordering[d]])

        # Get the vocabulary.
        self.vocab = [xx for xx in self.get_uniq_words()]

## Text Preprocessing

Before we build our information retrieval system, we need to understand **text preprocessing** — the steps we take to normalize text before indexing and searching.

Our IR system applies the following preprocessing pipeline to all documents and queries:

1. **Lowercase**: Convert all text to lowercase ("The" → "the")
2. **Tokenization**: Split text into individual words on whitespace
3. **Remove non-alphanumeric**: Strip punctuation and special characters
4. **Stemming**: Reduce words to their root form using the Porter Stemmer

### Why Stemming?

Stemming is crucial for information retrieval. Consider the query "running shoes":
- Without stemming: Only matches documents containing exactly "running"
- With stemming: "running" → "run", which also matches "runs", "ran", "runner"

This dramatically improves **recall** — our ability to find all relevant documents. The Porter Stemmer we use applies rules like:
- "running" → "run"
- "machines" → "machin"
- "learning" → "learn"

The preprocessing is already implemented in the `IRSystem` class above. When you build your inverted index, you'll be working with already-stemmed tokens.

### Part 1: Inverted Index

You will build an inverted index — a data structure that maps words to the documents in which they occur and how many times they occur. Unlike a positional index (which stores exact positions), we simply need to track the **term frequency** (count) of each word in each document.

**Structure:** `inv_index[word][doc_id] = term_count`

For example, if "drawer" appears 8 times in document 24, then: `inv_index["drawer"][24] = 8`

The documents will have already been read in at this point. The following instance variables are available: `self.titles` (a list of strings), `self.docs` (a list of lists of strings), and `self.vocab` (a list of strings).

In [ ]:
class IRSystem(IRSystem):
    def index(self):
        """
        Build an index of the documents.
        """
        print("Indexing...")
        # ------------------------------------------------------------------
        # TODO: Create an inverted index.
        #       This index should map words to documents and store the count
        #       of how many times each word appears in each document.
        #       Some helpful instance variables:
        #         * self.docs = List of documents
        #         * self.titles = List of titles

        # Note: To avoid having to initialize inv_index manually, we import
        # defaultdict from collections.
        
        # Structure: inv_index[word][doc_id] = term_count
        # Example: inv_index["drawer"] = {24: 8, 33: 2, 7: 1, ...}
        inv_index = defaultdict(lambda: defaultdict(int))
        
        # TODO: Generate inverted index here
        for i in range(len(self.docs)):
            pass

        self.inv_index = inv_index

        # ------------------------------------------------------------------

        # turn self.docs into a map from ID to bag of words
        id_to_bag_of_words = {}
        for d, doc in enumerate(self.docs):
            bag_of_words = set(doc)
            id_to_bag_of_words[d] = bag_of_words
        self.docs = id_to_bag_of_words

### Sanity Check: `index()`

In [ ]:
# === Sanity Check: Inverted Index ===
# Run this cell after implementing index() to verify your implementation

# Create a temporary test instance
_test_ir = IRSystem()
_test_ir.read_data('./data/ProjectGutenberg')
_test_ir.index()

# Check 1: inv_index should exist
assert hasattr(_test_ir, 'inv_index'), "ERROR: inv_index not found! Make sure you create self.inv_index"

# Check 2: A known word should be in the index
assert "drawer" in _test_ir.inv_index, "ERROR: 'drawer' not found in inv_index"

# Check 3: "drawer" should appear in doc 24 (Metamorphosis - Franz Kafka)
assert 24 in _test_ir.inv_index["drawer"], "ERROR: 'drawer' should appear in document 24"

# Check 4: "drawer" should appear 8 times in doc 24
count = _test_ir.inv_index["drawer"][24]
assert count == 8, f"ERROR: Expected count of 8 for 'drawer' in doc 24, got {count}"

# Check 5: "drawer" should appear 9 times in doc 33 (The Adventures of Sherlock Holmes)
count = _test_ir.inv_index["drawer"][33]
assert count == 9, f"ERROR: Expected count of 9 for 'drawer' in doc 33, got {count}"

print("All sanity checks passed for index()!")
del _test_ir  # Clean up

### Part 2: Get Posting

Now implement `get_posting` which returns the list of document IDs (sorted) in which a given word occurs. This uses your inverted index from Part 1.

In [ ]:
class IRSystem(IRSystem):
    def get_posting(self, word):
        """
        Given a word, this returns the list of document indices (sorted) in
        which the word occurs.
        """
        # ------------------------------------------------------------------
        # TODO: return the list of postings for a word.
        posting = []

        return posting
        # ------------------------------------------------------------------
    
    def get_posting_unstemmed(self, word):
        """
        Given a word, this *stems* the word and then calls get_posting on the
        stemmed word to get its postings list. You should *not* need to change
        this function. It is needed for submission.
        """
        word = self.p.stem(word)
        return self.get_posting(word)

In [ ]:
# === Sanity Check: get_posting() ===
# Run this cell after implementing get_posting() to verify your implementation

# Create a temporary test instance
_test_ir = IRSystem()
_test_ir.read_data('./data/ProjectGutenberg')
_test_ir.index()

# Check 1: "drawer" should appear in 18 documents
posting = _test_ir.get_posting("drawer")
assert len(posting) == 18, f"ERROR: 'drawer' should appear in 18 docs, got {len(posting)}"

# Check 2: Posting list should be sorted
assert posting == sorted(posting), "ERROR: Posting list should be sorted"

# Check 3: Should include doc 24 (Metamorphosis) and doc 33 (The Adventures of Sherlock Holmes)
assert 24 in posting, "ERROR: Doc 24 should be in posting list for 'drawer'"
assert 33 in posting, "ERROR: Doc 33 should be in posting list for 'drawer'"

# Check 4: First few docs should match expected
expected_start = [2, 7, 9, 10, 13]
assert posting[:5] == expected_start, f"ERROR: First 5 docs should be {expected_start}, got {posting[:5]}"

# Check 5: Non-existent word should return empty list
posting = _test_ir.get_posting("fizzbuzz")
assert posting == [], f"ERROR: 'fizzbuzz' not in corpus, should return [], got {posting}"

print("All sanity checks passed for get_posting()!")
del _test_ir  # Clean up

### Part 3: TF-IDF

Now we will compute and store the TF-IDF values. `compute_tfidf` computes and stores the TF-IDF values for words and documents. For this you will probably want to be aware of the class variable `self.vocab`, which holds the list of all unique words, as well as the inverted index you created earlier.

**Output structure:** `self.tfidf[word][doc] = tf_idf_value`

For example: `self.tfidf["drawer"] = {24: 0.912, 33: 1.021, ...}`

You must also implement `get_tfidf` to return the TF-IDF weight for a particular word and document ID. Make sure you correctly handle the case where the word isn't present in your vocabulary!

In [ ]:
class IRSystem(IRSystem):
    def compute_tfidf(self):
        """
        Compute and store TF-IDF values for words and documents.
        Recall: TF-IDF = (1 + log10(tf)) * log10(N/df)
        where tf = term frequency, N = number of documents, df = document frequency
        """
        print("Calculating tf-idf...")
        self.tfidf = {}
        
        # ------------------------------------------------------------------
        # TODO: Compute TF-IDF values for all word-document pairs.
        #       Store the results in self.tfidf as a nested dictionary:
        #       self.tfidf[word][doc_id] = tfidf_value
        #       
        #       Useful values:
        #         * self.vocab = list of all unique words
        #         * self.inv_index = inverted index (word -> doc -> count)
        #         * len(self.docs) = number of documents (N)

        # YOUR CODE HERE

        # ------------------------------------------------------------------

    def get_tfidf(self, word, document):
        """
        Return the TF-IDF weight for a word in a specific document.
        """
        # ------------------------------------------------------------------
        # TODO: Return the TF-IDF value for the given word and document index.
        #       Return 0 if the word is not in the vocabulary or not in
        #       the specified document.
        tfidf = 0.0
        # ------------------------------------------------------------------
        return tfidf

    def get_tfidf_unstemmed(self, word, document):
        """
        Given a word, this *stems* the word and then calls get_tfidf on the
        stemmed word to get its TF-IDF value. You should *not* need to change
        this function. It is needed for submission.
        """
        word = self.p.stem(word)
        return self.get_tfidf(word, document)

### Sanity Check: TF-IDF

In [ ]:
# === Sanity Check: TF-IDF ===
# Run this cell after implementing compute_tfidf() and get_tfidf()

# Create a temporary test instance
_test_ir = IRSystem()
_test_ir.read_data('./data/ProjectGutenberg')
_test_ir.index()
_test_ir.compute_tfidf()

epsilon = 1e-4

# Check 1: tfidf should exist
assert hasattr(_test_ir, 'tfidf'), "ERROR: tfidf not found! Make sure you create self.tfidf"

# Check 2: TF-IDF for "drawer" in doc 24 should be approximately 0.995
tfidf = _test_ir.get_tfidf("drawer", 24)
expected = 0.9950853045539215
assert abs(tfidf - expected) < epsilon, f"ERROR: TF-IDF for 'drawer' in doc 24 should be ~{expected}, got {tfidf}"

# Check 3: TF-IDF for "drawer" in doc 33 should be approximately 1.022
tfidf = _test_ir.get_tfidf("drawer", 33)
expected = 1.0218318713091326
assert abs(tfidf - expected) < epsilon, f"ERROR: TF-IDF for 'drawer' in doc 33 should be ~{expected}, got {tfidf}"

# Check 4: TF-IDF for non-existent word should return 0
tfidf = _test_ir.get_tfidf("fizzbuzz", 0)
assert tfidf == 0, f"ERROR: TF-IDF for non-existent word should be 0, got {tfidf}"

# Check 5: TF-IDF for word not in document should return 0
tfidf = _test_ir.get_tfidf("drawer", 0)
assert tfidf == 0, f"ERROR: TF-IDF for word not in document should be 0, got {tfidf}"

print("All sanity checks passed for compute_tfidf() and get_tfidf()!")
del _test_ir  # Clean up

### Part 4: Rank Retrieve

Now, we will implement `rank_retrieve`. This function returns a rank-ordered list of the top documents for a given query using **cosine similarity** between the query and document TF-IDF vectors.

**Cosine Similarity Formula:** `cosine(q, d) = (q · d) / (||q|| × ||d||)`

where `q` is the query vector, `d` is the document vector, and `||·||` denotes the L2 norm (length) of the vector.

You should:
1. Compute TF weights for query terms: `tf_query = 1 + log10(count)`
2. Use TF-IDF weights for document terms (from Part 3)
3. Normalize by both the query AND document vector lengths (L2 norms)

In [ ]:
class IRSystem(IRSystem):
    def rank_retrieve(self, query):
        """
        Given a query (a list of words), return a rank-ordered list of
        documents (by ID) and score for the query.
        """
        scores = [0.0 for xx in range(len(self.titles))]
        # ------------------------------------------------------------------
        # TODO: Implement cosine similarity between a document and a list of
        #       query words.

        # Right now, this code is a placeholder that simply gets the score by
        # taking the Jaccard similarity between the query and every document. 
        # Please replace with your implementation of cosine similarity-based 
        # retrieval.
        # HINT: you can make use of Counter from collections
        words_in_query = set()
        for word in query:
            words_in_query.add(word)

        for d, words_in_doc in self.docs.items():
            scores[d] = len(words_in_query.intersection(words_in_doc)) \
                /float(len(words_in_query.union(words_in_doc)))
        # ------------------------------------------------------------------

        ranking = [idx for idx, sim in sorted(enumerate(scores),
                                              key=lambda xx: xx[1],
                                              reverse=True)]
        results = []
        for i in range(10):
            results.append((ranking[i], scores[ranking[i]]))
        return results

### Sanity Check: `rank_retrieve()`

In [ ]:
# === Sanity Check: rank_retrieve() (Cosine Similarity) ===
# Run this cell after implementing rank_retrieve() to verify your implementation

# Create a temporary test instance
_test_ir = IRSystem()
_test_ir.read_data('./data/ProjectGutenberg')
_test_ir.index()
_test_ir.compute_tfidf()

# Helper to stem a query string
def stem_query(ir, query_str):
    query = query_str.lower().split()
    return [ir.p.stem(word) for word in query]

epsilon = 1e-4

# Check 1: "chest of drawers" top result should be doc 24 (Metamorphosis)
query = stem_query(_test_ir, "chest of drawers")
results = _test_ir.rank_retrieve(query)
top_doc, top_score = results[0]
assert top_doc == 24, f"ERROR: Top doc for 'chest of drawers' should be 24, got {top_doc}"

# Check 2: Score should be approximately 0.0331 (vanilla cosine)
expected_score = 0.033118100770043575
assert abs(top_score - expected_score) < epsilon, f"ERROR: Score should be ~{expected_score}, got {top_score}"

# Check 3: "pumpkin pies" top result should be doc 41 (Legend of Sleepy Hollow)
query = stem_query(_test_ir, "pumpkin pies")
results = _test_ir.rank_retrieve(query)
top_doc, top_score = results[0]
assert top_doc == 41, f"ERROR: Top doc for 'pumpkin pies' should be 41, got {top_doc}"

# Check 4: Score should be approximately 0.0777 (vanilla cosine)
expected_score = 0.07769755389348589
assert abs(top_score - expected_score) < epsilon, f"ERROR: Score should be ~{expected_score}, got {top_score}"

# Check 5: Results should be in descending order by score
query = stem_query(_test_ir, "the cold breeze")
results = _test_ir.rank_retrieve(query)
scores = [score for doc, score in results]
assert scores == sorted(scores, reverse=True), "ERROR: Results should be sorted by score descending"

print("All sanity checks passed for rank_retrieve()!")
del _test_ir  # Clean up

### Part 5: Dense Retrieval

Modern information retrieval systems often use **dense retrieval** — representing documents and queries as dense vectors (embeddings) from neural language models like BERT, rather than sparse TF-IDF vectors.

We have pre-computed BERT embeddings for all 60 books and the development queries. These embeddings capture semantic meaning, so documents about similar topics will have similar embeddings even if they don't share exact words.

Your task:
1. Implement `cosine_similarity_dense` to compute cosine similarity between two dense vectors
2. Implement `dense_retrieve` to return the top-k documents most similar to a query embedding

In [ ]:
import numpy as np

# Load pre-computed BERT embeddings
# doc_embeddings: shape (60, 384) - one embedding per book
# query_embeddings: shape (num_queries, 384) - one embedding per dev query
# query_texts: list of query strings corresponding to query_embeddings

doc_embeddings = np.load('./data/embeddings/doc_embeddings.npy')
query_embeddings = np.load('./data/embeddings/query_embeddings.npy')
with open('./data/embeddings/query_texts.txt', 'r') as f:
    query_texts = [line.strip() for line in f.readlines()]

print(f"Loaded {doc_embeddings.shape[0]} document embeddings of dimension {doc_embeddings.shape[1]}")
print(f"Loaded {query_embeddings.shape[0]} query embeddings")

In [ ]:
class IRSystem(IRSystem):
    def cosine_similarity_dense(self, vec1, vec2):
        """
        Compute the cosine similarity between two dense vectors.
        
        Args:
            vec1: numpy array of shape (d,)
            vec2: numpy array of shape (d,)
        
        Returns:
            float: cosine similarity between vec1 and vec2
        """
        # ------------------------------------------------------------------
        # TODO: Implement cosine similarity for dense vectors.
        #       Hint: This should be ~1-2 lines of code using numpy operations.
        #       Remember: cosine_similarity = (a · b) / (||a|| * ||b||)
        
        return 0.0
        # ------------------------------------------------------------------


    def dense_retrieve(self, query_embedding, doc_embeddings, titles, k=5):
        """
        Given a query embedding and document embeddings, return the top-k 
        most similar documents.
        
        Args:
            query_embedding: numpy array of shape (d,) - the query vector
            doc_embeddings: numpy array of shape (n, d) - all document vectors
            titles: list of document titles (length n)
            k: number of top documents to return
        
        Returns:
            list of tuples: [(doc_id, title, score), ...] for top-k documents,
                            sorted by similarity score (highest first)
        """
        # ------------------------------------------------------------------
        # TODO: Implement dense retrieval.
        #       1. Compute cosine similarity between query and all documents
        #       2. Find the top-k documents by similarity
        #       3. Return list of (doc_id, title, score) tuples
        #       Hint: Consider using np.argsort() and remember to sort descending!
        
        results = []
        
        return results
        # ------------------------------------------------------------------

### Sanity Check: Dense Retrieval

In [ ]:
# === Sanity Check: Dense Retrieval ===
# Run this cell after implementing cosine_similarity_dense() and dense_retrieve()

# Create a temporary test instance
_test_ir = IRSystem()
_test_ir.read_data('./data/ProjectGutenberg')
_test_ir.index()

epsilon = 1e-4

# Check 1: Cosine similarity of identical vectors should be 1.0
vec1 = np.array([1.0, 0.0, 0.0])
vec2 = np.array([1.0, 0.0, 0.0])
sim = _test_ir.cosine_similarity_dense(vec1, vec2)
assert abs(sim - 1.0) < epsilon, f"ERROR: Cosine similarity of identical vectors should be 1.0, got {sim}"

# Check 2: Cosine similarity of orthogonal vectors should be 0.0
vec1 = np.array([1.0, 0.0, 0.0])
vec2 = np.array([0.0, 1.0, 0.0])
sim = _test_ir.cosine_similarity_dense(vec1, vec2)
assert abs(sim - 0.0) < epsilon, f"ERROR: Cosine similarity of orthogonal vectors should be 0.0, got {sim}"

# Check 3: Cosine similarity of opposite vectors should be -1.0
vec1 = np.array([1.0, 0.0, 0.0])
vec2 = np.array([-1.0, 0.0, 0.0])
sim = _test_ir.cosine_similarity_dense(vec1, vec2)
assert abs(sim - (-1.0)) < epsilon, f"ERROR: Cosine similarity of opposite vectors should be -1.0, got {sim}"

# Check 4: dense_retrieve for query 0 ("chest of drawers") should return doc 14 as top result
results = _test_ir.dense_retrieve(query_embeddings[0], doc_embeddings, _test_ir.titles, k=1)
top_doc_id, top_title, top_score = results[0]
assert top_doc_id == 14, f"ERROR: Top doc for query 0 should be 14, got {top_doc_id}"

# Check 5: Score should be approximately 0.2902
expected_score = 0.29019150137901306
assert abs(top_score - expected_score) < epsilon, f"ERROR: Score should be ~{expected_score}, got {top_score}"

# Check 6: Results should have correct format (doc_id, title, score)
results = _test_ir.dense_retrieve(query_embeddings[0], doc_embeddings, _test_ir.titles, k=3)
assert len(results) == 3, f"ERROR: Should return 3 results, got {len(results)}"
assert isinstance(results[0][0], (int, np.integer)), "ERROR: First element of tuple should be int (doc_id)"
assert isinstance(results[0][1], str), "ERROR: Second element of tuple should be str (title)"
assert isinstance(results[0][2], (float, np.floating)), "ERROR: Third element of tuple should be float (score)"

print("All sanity checks passed for dense retrieval!")
del _test_ir  # Clean up

Let's also add a few methods that given a string, will process and then return the list of matching documents for the different methods you have implemented. You do not need to add any additional code here.

In [ ]:
class IRSystem(IRSystem): 
    def process_query(self, query_str):
        """
        Given a query string, process it and return the list of lowercase,
        alphanumeric, stemmed words in the string.
        """
        # make sure everything is lower case
        query = query_str.lower()
        # split on whitespace
        query = query.split()
        # remove non alphanumeric characters
        query = [self.alphanum.sub('', xx) for xx in query]
        # stem words
        query = [self.p.stem(xx) for xx in query]
        return query

    def query_rank(self, query_str):
        """
        Given a string, process and then return the list of the top matching
        documents, rank-ordered.
        """
        query = self.process_query(query_str)
        return self.rank_retrieve(query)

In [ ]:
irsys = IRSystem()
irsys.read_data('./data/ProjectGutenberg')
irsys.index()
irsys.compute_tfidf()

Now let's compare your TF-IDF cosine similarity results with dense retrieval results on the same queries. This will help you see how the two approaches differ in what they consider "similar".

In [ ]:
def compare_retrieval_methods(irsys, query_text, query_embedding, doc_embeddings, k=5):
    """
    Compare TF-IDF and dense retrieval results for the query "machine learning is cool".
    """
    print("=" * 80)
    print("COMPARING TF-IDF vs DENSE RETRIEVAL")
    print("=" * 80)
    
    print(f"\nQuery: '{query_text}'")
    print("-" * 40)
    
    # TF-IDF results
    tfidf_results = irsys.query_rank(query_text)[:k]
    print(f"\nTF-IDF Top-{k}:")
    for rank, (doc_id, score) in enumerate(tfidf_results, 1):
        print(f"  {rank}. {irsys.titles[doc_id]} (score: {score:.4f})")
    
    # Dense retrieval results
    dense_results = irsys.dense_retrieve(query_embedding, doc_embeddings, irsys.titles, k=k)
    print(f"\nDense Retrieval Top-{k}:")
    for rank, (doc_id, title, score) in enumerate(dense_results, 1):
        print(f"  {rank}. {title} (score: {score:.4f})")
    print("=" * 80)

# Run the comparison with just "machine learning is cool"
compare_retrieval_methods(irsys, "machine learning is cool", query_embeddings[1], doc_embeddings)

### Part 6: Comparing Retrieval Methods

Look at the results above for the query "machine learning is cool".

**Your task:** In 3-5 sentences, reflect on the differences between TF-IDF and dense retrieval. What are the strengths and weaknesses of each approach? When might you prefer one over the other?

*Hint: Think about the difference between **lexical matching** (matching based on exact words) and **semantic matching** (matching based on meaning). How does each method decide what counts as a "match"?*

In [ ]:
class IRSystem(IRSystem):
    def reflection_on_retrieval_methods(self):
        # TODO: Place your response into the response string below
        response = ""
        return response

### Part 7: Running the Full IR System

You can use the `run_tests` and `run_query` functions to test your code. `run_tests` tests how different components of your search engine code perform on a small set of queries and checks whether or not it matches up with our solution's results. `run_query` can be used to test your code on individual queries. 

Note that the first run for either of these functions might take a little while, since it will stem all the words in every document and create a directory named `stemmed/` in `../data/ProjectGutenberg/`. This is meant to be a simple cache for the stemmed versions of the text documents. Later runs will be much faster after the first run since all the stemming will already be completed. However, this means that **if something happens during this first run and it does not get through processing all the documents, you may be left with an incomplete set of documents in `../data/ProjectGutenberg/stemmed/`. If this happens, simply remove the `stemmed/` directory and re-run!**

In [ ]:
def run_tests(irsys):
    print("===== Running tests =====")

    ff = open('./data/dev_queries.txt')
    questions = [xx.strip() for xx in ff.readlines()]
    ff.close()
    ff = open('./data/dev_solutions.txt')
    solutions = [xx.strip() for xx in ff.readlines()]
    ff.close()

    epsilon = 1e-4
    for part in range(5):
        points = 0
        num_correct = 0
        num_total = 0

        prob = questions[part]
        soln = json.loads(solutions[part])
        

        if part == 0:  # inverted index test (counts)
            print("Inverted Index Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                # Get count from inverted index
                word_stemmed = irsys.p.stem(word)
                count = irsys.inv_index.get(word_stemmed, {}).get(doc, 0)
                if count == soln[i]:
                    num_correct += 1

        elif part == 1:  # get postings test
            print("Get Postings Test")
            words = prob.split(", ")
            for i, word in enumerate(words):
                num_total += 1
                posting = irsys.get_posting_unstemmed(word)
                if posting == soln[i]:
                    num_correct += 1

        elif part == 2:  # tfidf test
            print("TF-IDF Test")
            queries = prob.split("; ")
            queries = [xx.split(", ") for xx in queries]
            queries = [(xx[0], int(xx[1])) for xx in queries]
            for i, (word, doc) in enumerate(queries):
                num_total += 1
                guess = irsys.get_tfidf_unstemmed(word, doc)
                if guess >= float(soln[i]) - epsilon and \
                        guess <= float(soln[i]) + epsilon:
                    num_correct += 1

        elif part == 3:  # cosine similarity test
            print("Cosine Similarity Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                ranked = irsys.query_rank(query)
                top_rank = ranked[0]
                if top_rank[0] == soln[i][0]:
                    if top_rank[1] >= float(soln[i][1]) - epsilon and \
                            top_rank[1] <= float(soln[i][1]) + epsilon:
                        num_correct += 1

        elif part == 4:  # dense retrieval test
            print("Dense Retrieval Test")
            queries = prob.split(", ")
            for i, query in enumerate(queries):
                num_total += 1
                results = irsys.dense_retrieve(query_embeddings[i], doc_embeddings, irsys.titles, k=1)
                if len(results) > 0:
                    top_result = results[0]  # (doc_id, title, score)
                    if top_result[0] == soln[i][0]:
                        if top_result[2] >= float(soln[i][1]) - epsilon and \
                                top_result[2] <= float(soln[i][1]) + epsilon:
                            num_correct += 1

        feedback = "%d/%d Correct. Accuracy: %f" % \
                   (num_correct, num_total, float(num_correct) / num_total)

        if part == 1:
            if num_correct == num_total:
                points = 2
            elif num_correct >= 0.5 * num_total:
                points = 1
            else:
                points = 0
        else:
            if num_correct == num_total:
                points = 3
            elif num_correct > 0.75 * num_total:
                points = 2
            elif num_correct > 0:
                points = 1
            else:
                points = 0

        print("    Score: %d Feedback: %s" % (points, feedback))

In [ ]:
## Run this cell to run the tests
run_tests(irsys)

In [ ]:
def run_query(query):
    irsys = IRSystem()
    irsys.read_data('./data/ProjectGutenberg')
    irsys.index()
    irsys.compute_tfidf()
    
    print("Best matching documents to '%s':" % query)
    results = irsys.query_rank(query)
    for docId, score in results:
        print("%s: %e" % (irsys.titles[docId], score))
        
# Run any query you want!
run_query("My very own query")

For reference, you can run the cell below to see the titles of all the documents in our dataset. As sanity checks, you can try tailoring your queries in `run_query` to output certain titles and/or checking what IR system outputs against the list of titles to see if the results make sense (i.e. the book _Great Pianists on Piano Playing_ should probably be among the top results if the query is "pianists play piano").

In [ ]:
# Prints full list of book titles in dataset (in alphabetical order)
for i in range(len(irsys.titles)):
    print("%d. %s" % (i + 1, irsys.titles[i]))

### Part 8 (Ethics): Information Retrieval Systems and Search Engines

Safiya Umoja Noble's <a href="https://nyupress.org/9781479837243/algorithms-of-oppression/">Algorithms of Oppression</a> (2018) provides insight into how search engines and information retrieval algorithms can exhibit substantial racist and sexist biases. Noble demonstrates how prejudice against black women is embedded into search engine ranked results. These biases are apparent in both Google search's autosuggestions and the first page of Google results. In this assignment, we have explored numerous features that are built into information retrieval systems, like Google Search. 

How could the algorithms you built in this assignment contribute to enforcing real-world biases? 

How can we reduce data discrimination and algorithmic bias that perpetuate gender and racial inequalities in search results and IR system?

Please provide a short response to each of these questions (1-2 paragraphs per question).

In [ ]:
class IRSystem(IRSystem):
    def algorithmic_bias_in_IR_systems(self):
        # TODO: Place your response into the response string below
        response = ""
        return response

Once you're ready to submit, you can run the cell below to prepare and zip
up your solution:

In [ ]:
%%bash

if [[ ! -f "./pa3.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. This probably means you're running on Google Colab. You'll need to go to File->Download .ipynb to download your notebook and other files, then zip them locally. See the README for more information."
else
    echo "Found notebook file, creating submission zip..."
    rm -f submission.zip
    zip -r submission.zip pa3.ipynb deps/
fi

If you're running on Google Colab, see the README for instructions on
how to submit.

__Best of luck!__

__Some reminders for submission:__
* If you have any extra files required for your implementation to work, make
 sure they are in a `deps/` folder on the same level as `pa3.ipynb` and
 include that folder in your submission zip file.
 * Make sure you didn't accidentally change the name of your notebook file,
 (it should be `pa3.ipynb`) as that is required for the autograder to work.
* Go to Gradescope (gradescope.com), find the PA3 IR assignment and
upload your zip file (`submission.zip`) as your solution.
* Wait for the autograder to run (it should only take a minute or so) and check
that your submission was graded successfully! If the autograder fails, or you
get an unexpected score it may be a sign that your zip file was incorrect.